In [14]:
# Import necessary packages
using PyPlot
using SparseArrays
using LinearAlgebra
using IterativeSolvers
using AlgebraicMultigrid
using Printf
using WriteVTK

In [19]:
struct Grid
    x::Vector{Float64}
    y::Vector{Float64}
    z::Vector{Float64}    
    nx::Int64
    ny::Int64
    nz::Int64
    LX::Float64
    LY::Float64
    LZ::Float64
    dx::Float64
    dy::Float64
    dz::Float64
    solution::Array{Float64,3}
    rhs::Array{Float64,3}
    residual::Array{Float64,3}
    function Grid(W,H,D,nx,ny,nz)
        x = LinRange(0,W,nx)
        y = LinRange(0,H,ny)
        z = LinRange(0,D,nz)
        dx = x[2]-x[1]
        dy = y[2]-y[1]
        dz = z[2]-z[1]
        
        return new(x,y,z,nx,ny,nz,W,H,D,dx,dy,dz,zeros(nz,ny,nx),zeros(nz,ny,nx),zeros(nz,ny,nx))
    end
end

function poisson_smoother!(grid::Grid,cycles::Int64)
    coef_center = -2/grid.dx^2 -2/grid.dy^2 - 2/grid.dz^2
    coef_right = 1/grid.dx^2
    coef_left = 1/grid.dx^2
    coef_up = 1/grid.dy^2
    coef_down = 1/grid.dy^2
    coef_front = 1/grid.dz^2
    coef_back = 1/grid.dz^2
    theta = 1.75;
    cycle = 1
    while cycle <= cycles
        #red-black update        
        for j in 2:(grid.nx-1)
            for i in 2:(grid.ny-1) # julia is column-major - inner loop over columns
                for k in 2:(grid.nz-1)
                    # in Gauss-Seidel iterations, the solution (s) is updated within each iteration.
                    # note that the 1.0 is the (uniform) value of the right hand side, hard-coded for speed.
                    this_residual = grid.rhs[k,i,j] - coef_left*grid.solution[k,i,j-1] - coef_right*grid.solution[k,i,j+1] - coef_up*grid.solution[k,i-1,j] - coef_down*grid.solution[k,i+1,j] - coef_center*grid.solution[k,i,j] - coef_front*grid.solution[k-1,i,j]-coef_back*grid.solution[k+1,i,j]
                    grid.solution[k,i,j] += theta*this_residual/coef_center
                end
            end            
        end
        cycle += 1
    end
    Threads.@threads for j in 2:(grid.nx-1)
         for i in 2:(grid.ny-1) # julia is column-major - inner loop over columns
            for k in 2:(grid.nz-1)
                 # in Gauss-Seidel iterations, the solution (s) is updated within each iteration.
                 # note that the 1.0 is the (uniform) value of the right hand side, hard-coded for speed.
                 grid.residual[k,i,j] = grid.rhs[k,i,j] - coef_left*grid.solution[k,i,j-1] - coef_right*grid.solution[k,i,j+1] - coef_up*grid.solution[k,i-1,j] - coef_down*grid.solution[k,i+1,j] - coef_center*grid.solution[k,i,j] - coef_front*grid.solution[k-1,i,j]-coef_back*grid.solution[k+1,i,j]
            end
         end
    end
end

function poisson_restriction!(fine::Grid,coarse::Grid,mgunit::Int64)
    # restriction (fine) -> (coarse)
    coarse.rhs[:,:,:] = fine.residual[1:mgunit:end,1:mgunit:end,1:mgunit:end]
end

function poisson_prolongation(fine::Grid,coarse::Grid,mgunit::Int64)
    dsol = zeros(fine.nz,fine.ny,fine.nx)
    # use bilinear interpolation to go from coarse to fine
    local coarse_j::Int64
    local coarse_i::Int64
    local coarse_k::Int64
    local dx::Float64
    local dy::Float64
    local dz::Float64
    for j in 2:fine.nx-1
        coarse_j = Int64( floor( fine.x[j]/coarse.dx )+1 )
        dx = (fine.x[j]-coarse.x[coarse_j])/coarse.dx
        
        for i in 2:fine.ny-1
            coarse_i = Int64( floor( fine.y[i]/coarse.dy) + 1)
            dy = (fine.y[i]-coarse.y[coarse_i]) / coarse.dy
            
            for k in 2:fine.nz-1
                coarse_k = Int64( floor( fine.z[k]/coarse.dz) + 1 )
                dz = (fine.z[k]-coarse.z[coarse_k]) / coarse.dz
                                
                dsol[k,i,j] += (1.0-dz)*(1.0-dx)*(1.0-dy)*coarse.solution[coarse_k,coarse_i,coarse_j]
                dsol[k,i,j] += (1.0-dz)*(1.0-dx)*(dy)*coarse.solution[coarse_k,coarse_i+1,coarse_j]
                dsol[k,i,j] += (1.0-dz)*(dx)*(1.0-dy)*coarse.solution[coarse_k,coarse_i,coarse_j+1]
                dsol[k,i,j] += (1.0-dz)*(dx)*(dy)*coarse.solution[coarse_k,coarse_i+1,coarse_j+1]
            
                dsol[k,i,j] += (dz)*(1.0-dx)*(1.0-dy)*coarse.solution[coarse_k+1,coarse_i,coarse_j]
                dsol[k,i,j] += (dz)*(1.0-dx)*(dy)*coarse.solution[coarse_k+1,coarse_i+1,coarse_j]
                dsol[k,i,j] += (dz)*(dx)*(1.0-dy)*coarse.solution[coarse_k+1,coarse_i,coarse_j+1]
                dsol[k,i,j] += (dz)*(dx)*(dy)*coarse.solution[coarse_k+1,coarse_i+1,coarse_j+1]
            end
        end
    end
    return dsol
end

poisson_prolongation (generic function with 1 method)

In [20]:
# code to write a .vtr file:
function visualization(grid::Grid; filename="test.vtr")
    # write the visualization output from the regular grid as a .vts file.
    vtk_grid(filename, grid.x, grid.y, grid.z) do vtk
        vtk["phi"] = grid.solution
        vtk["residual"] = grid.residual
    end
end

visualization (generic function with 1 method)

In [21]:
# define an indexing function that maps from (i,j) node numbering to an integer index
nx = 257
ny = 257
nz = 257
width = 1e3
height = 1.5e3
depth = 1.25e3

# Define levels of grid
nlevel=6
mgunit=2
grids = Vector{Grid}() # grids to use with multigrid solver
cycles = Vector{Int64}() # number of gauss-seidel iterations to take at each multigrid level
for level in 1:nlevel
    level_nx = Int64((nx-1)/mgunit^(level-1)+1)
    level_ny = Int64((ny-1)/mgunit^(level-1)+1)
    level_nz = Int64((nz-1)/mgunit^(level-1)+1)
    println(level_nx,' ', level_ny,' ',level_nz)
    push!(grids, Grid(width,height,depth,level_nx,level_ny,level_nz) )
    push!(cycles,2^(level+1)*5)
end

# set RHS for finest grid.
level = 1
for j in 2:grids[level].nx-1
    for i in 2:grids[level].ny-1
        for k in 2:grids[level].nz-1
            grids[level].rhs[k,i,j] = 1.0        
        end
    end
end

function multigrid_smoother!(grids::Vector{Grid},cycles::Vector{Int64},outer_cycles::Int64)
    level = 1
    cycle = 1
    # outer solver iterations
    poisson_smoother!(grids[1],0) # compute starting residual
    println(norm(grids[1].residual))
    while cycle <= outer_cycles
        for level in 1:nlevel-1
            if cycle==1 || level > 1 # don't re-run smoother at first level if we're past first cycle.
                poisson_smoother!(grids[level],cycles[level])     
            end
            poisson_restriction!(grids[level],grids[level+1],mgunit)
            #println(norm(grids[level].residual))
        end
        # end at grid level nlevel-1
        # compute correction to current solution using a direct solver
        # interpolate corrections onto finer grids
        for level in nlevel:-1:2
            poisson_smoother!(grids[level],cycles[level])
            println(norm(grids[level].residual))
            dsol = poisson_prolongation(grids[level-1],grids[level],mgunit)
            grids[level-1].solution[:,:,:] += dsol
            # interpolate correction from level+1 onto level
        end
        poisson_smoother!(grids[1],cycles[1]) # run smoother on final (finest) level.  
        println("Cycle: ",cycle," Residual: ",norm(grids[1].residual))
        cycle += 1 # increment mg cycle counter
    end

end

@time multigrid_smoother!(grids,cycles,10)
visualization(grids[1],"test.vtr")

257 257 257
129 129 129
65 65 65
33 33 33
17 17 17
9 9 9
4072.0234527811845
7.195751805247488e-35
5.28826762189299e-19
3.06277106997723e-12
0.026064626904493494
0.32175193122222967
Cycle: 1 Residual: 4.495345988710504
2.082984158763465e-34
1.644374112008982e-18
8.736698114538585e-12
0.05214179195053081
0.37807220960764754
Cycle: 2 Residual: 3.0069695642498884
2.454368118700532e-34
1.6764715810331503e-18
8.422450634259428e-12
0.026109846207788456
0.06106845311503858
Cycle: 3 Residual: 0.17395308576691493
5.829856511851134e-35
5.365404022879292e-19
2.788064664751448e-12
5.151610695533636e-5
0.0002749022816691169
Cycle: 4 Residual: 0.002932741067690726
2.2718652317029178e-37
1.58392645281046e-21
7.804441565944164e-15
1.740227074370019e-5
2.6717209189257918e-5
Cycle: 5 Residual: 0.0002736212921653943
4.941159538911312e-38
3.5216844684654147e-22
1.826988571798764e-15
1.3700881206556837e-6
3.1283771698018624e-6
Cycle: 6 Residual: 1.064603988115246e-5
2.6800683031591532e-39
2.682638212050127e

LoadError: MethodError: no method matching visualization(::Grid, ::String)
[0mClosest candidates are:
[0m  visualization(::Grid; filename) at In[20]:2

In [18]:
visualization(grids[1] ; filename="test.vtr")

1-element Vector{String}:
 "test.vtr"